In [11]:
from PIL import Image
import os

def check_images(folder):
    removed = 0
    for root, dirs, files in os.walk(folder):
        for f in files:
            path = os.path.join(root, f)
            try:
                img = Image.open(path)
                img.verify()  # verify only
            except:
                print("Removing corrupted image:", path)
                os.remove(path)
                removed += 1
    print(f"Total corrupted images removed: {removed}")

# Run this on train and val folders
check_images("zDatasets/train")
check_images("zDatasets/val")


Removing corrupted image: zDatasets/train\brain_mri\p-99-Copy-Kopyala-_jpg.rf.a59db64afdc73faf72da2e44d4f7bcc9.jpg
Removing corrupted image: zDatasets/train\brain_mri\yw54y_jpeg.rf.043b8610ad8d4eae2e9aa85471b1f71d.jpg
Total corrupted images removed: 2
Removing corrupted image: zDatasets/val\brain_mri\p-94-Copy-Kopyala-_jpg.rf.870ae94327a63d6403bd7c1eed95b498.jpg
Total corrupted images removed: 1


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from PIL import Image

# ------------------------------
# Step 1: Set directories and parameters
# ------------------------------
train_dir = "zDatasets/train"  # contains brain_mri and not_brain
val_dir = "zDatasets/val"      # contains brain_mri and not_brain
img_size = (128, 128)
batch_size = 32
epochs = 5  # can increase for better results

# ------------------------------
# Step 2: Data generators with augmentation
# ------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Standard flow_from_directory generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# ------------------------------
# Step 3: Build EfficientNetB0 model
# ------------------------------
base_model = MobileNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(128,128,3)
)

# Fine-tune last layers
for layer in base_model.layers[-30:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

validator_model = Model(inputs=base_model.input, outputs=output)
validator_model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ------------------------------
# Step 4: Callbacks
# ------------------------------
checkpoint = ModelCheckpoint(
    "brain_mri_validator_full.h5",  # saving full model
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# ------------------------------
# Step 5: Training steps
# ------------------------------
steps_per_epoch = int(np.ceil(train_generator.samples / batch_size))
validation_steps = int(np.ceil(val_generator.samples / batch_size))

# ------------------------------
# Step 6: Train the model
# ------------------------------
history = validator_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=[checkpoint, early_stop],
    verbose=2
)

# ------------------------------
# Step 7: Save final weights (optional)
# ------------------------------
validator_model.save("brain_mri_validator_final_model.h5")
print("Training complete. Full model and weights saved!")

# ------------------------------
# Step 8: Inference for new images
# ------------------------------


# Example usage:
# result, score = is_brain_mri("test_image.jpg")
# print(result, score)


Found 12133 images belonging to 2 classes.
Found 5213 images belonging to 2 classes.
Epoch 1/5

Epoch 1: val_accuracy improved from -inf to 0.98945, saving model to brain_mri_validator_full.h5
380/380 - 82s - loss: 0.1235 - accuracy: 0.9604 - val_loss: 0.0426 - val_accuracy: 0.9894 - 82s/epoch - 216ms/step
Epoch 2/5

Epoch 2: val_accuracy improved from 0.98945 to 0.99751, saving model to brain_mri_validator_full.h5
380/380 - 76s - loss: 0.0158 - accuracy: 0.9972 - val_loss: 0.0124 - val_accuracy: 0.9975 - 76s/epoch - 200ms/step
Epoch 3/5

Epoch 3: val_accuracy improved from 0.99751 to 0.99885, saving model to brain_mri_validator_full.h5
380/380 - 69s - loss: 0.0078 - accuracy: 0.9987 - val_loss: 0.0047 - val_accuracy: 0.9988 - 69s/epoch - 181ms/step
Epoch 4/5

Epoch 4: val_accuracy improved from 0.99885 to 0.99981, saving model to brain_mri_validator_full.h5
380/380 - 70s - loss: 0.0044 - accuracy: 0.9993 - val_loss: 0.0019 - val_accuracy: 0.9998 - 70s/epoch - 185ms/step
Epoch 5/5

Epo

In [7]:
import os

train_brain_dir = "zDatasets/train/brain_mri"
train_not_brain_dir = "zDatasets/train/not_brain"
val_brain_dir = "zDatasets/val/brain_mri"
val_not_brain_dir = "zDatasets/val/not_brain"

def count_images(folder):
    return sum([len(files) for _, _, files in os.walk(folder)])

print("Train Brain MRI images:", count_images(train_brain_dir))
print("Train Not Brain images:", count_images(train_not_brain_dir))
print("Validation Brain MRI images:", count_images(val_brain_dir))
print("Validation Not Brain images:", count_images(val_not_brain_dir))


Train Brain MRI images: 5666
Train Not Brain images: 6469
Validation Brain MRI images: 1725
Validation Not Brain images: 3489


In [43]:
from PIL import Image
import numpy as np

def load_and_preprocess_image(file_path, target_size=(128,128)):
    """Load image safely and preprocess for model input."""
    try:
        img = Image.open(file_path).convert('RGB')
        img = img.resize(target_size)
        img_array = np.array(img) / 255.0
        if img_array.shape != (target_size[0], target_size[1], 3):
            print(f"Skipped image with wrong shape: {file_path}")
            return None
        return np.expand_dims(img_array, axis=0)
    except Exception as e:
        print(f"Corrupted or unreadable image skipped: {file_path}, Error: {e}")
        return None

def is_brain_mri(file_path, threshold=0.5, model=validator_model):
    """Predict whether the input image is a brain MRI."""
    img_array = load_and_preprocess_image(file_path)
    if img_array is None:
        return False, 0.0
    
    pred = float(model.predict(img_array, verbose=0)[0][0])
    
    # brain_mri = 0 → pred close to 0 means brain MRI
    brain_mri = pred < threshold
    confidence = 1 - pred if brain_mri else pred  # confidence in correct class
    
    return brain_mri, confidence

# -------------------- Example usage --------------------
test_file = "E:/Final_year_project/brain_tumor_images/Training/pituitary/923.jpg"
brain_mri, confidence = is_brain_mri(test_file)

if brain_mri:
    print(f"Accepted: Brain MRI detected with confidence {confidence:.4f}")
else:
    print(f"Rejected: Not a brain MRI. Confidence {confidence:.4f}")


Accepted: Brain MRI detected with confidence 1.0000


In [40]:
print(train_generator.class_indices)


{'brain_mri': 0, 'not_brain': 1}


In [14]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import os

# ---------------- CONFIG ----------------
train_dir = "E:/Final_year_project/zDatasets/train/brain_mri"
img_size = (128, 128)
batch_size = 16
num_epochs = 5
learning_rate = 1e-4

# ---------------- DATA GENERATOR ----------------
train_datagen = ImageDataGenerator(rescale=1./255)
# Create a generator that also returns dummy labels
def generator_with_labels(gen):
    for batch in gen:
        yield batch, np.zeros((batch.shape[0], 128))  # dummy labels same size as output embedding

base_gen = train_datagen.flow_from_directory(
    directory=os.path.dirname(train_dir),
    classes=[os.path.basename(train_dir)],
    target_size=img_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=True
)

train_generator = generator_with_labels(base_gen)

# ---------------- MODEL ----------------
input_tensor = Input(shape=(img_size[0], img_size[1], 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
output = Dense(128, activation='linear')(x)  # embeddings
model = Model(inputs=input_tensor, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='mse')

# ---------------- TRAIN ----------------
history = model.fit(train_generator, steps_per_epoch=len(base_gen), epochs=num_epochs)
model.save("brain_one_class_resnet.h5")
print("One-class model saved!")


Found 5664 images belonging to 1 classes.
Epoch 1/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 125s 318ms/step - loss: 0.0294
Epoch 2/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 110s 311ms/step - loss: 2.7914e-04
Epoch 3/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 114s 322ms/step - loss: 1.7503e-04
Epoch 4/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 107s 301ms/step - loss: 1.2785e-04
Epoch 5/5
354/354 ━━━━━━━━━━━━━━━━━━━━ 106s 301ms/step - loss: 1.0245e-04


One-class model saved!


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ---------------- CONFIG ----------------
train_dir = "E:/Final_year_project/zDatasets/train/brain_mri"  # training images used for one-class
val_dir = "E:/Final_year_project/zDatasets/val"  # folder with test images (brain and non-brain)
img_size = (128, 128)
batch_size = 16
saved_model_path = "brain_one_class_resnet.h5"
percentile_threshold = 95  # percentile of distances to set anomaly threshold

# ---------------- LOAD MODEL ----------------
model = load_model(saved_model_path, compile=False)

# ---------------- DATA GENERATOR ----------------
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    directory=os.path.dirname(train_dir),
    classes=[os.path.basename(train_dir)],
    target_size=img_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

# ---------------- COMPUTE TRAINING EMBEDDINGS ----------------
print("Computing training embeddings...")
train_embeddings = []
for batch in train_gen:
    emb = model.predict(batch)
    train_embeddings.append(emb)
    if len(train_embeddings)*batch_size >= train_gen.n:
        break

train_embeddings = np.vstack(train_embeddings)
mean_embedding = np.mean(train_embeddings, axis=0)

# ---------------- SET THRESHOLD ----------------
distances = np.linalg.norm(train_embeddings - mean_embedding, axis=1)
threshold = np.percentile(distances, percentile_threshold)
print(f"Anomaly threshold set at {threshold:.4f}")

# ---------------- INFERENCE FUNCTION ----------------


Found 5664 images belonging to 1 classes.
Computing training embeddings...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317

In [16]:
import os 
def is_brain(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    emb = model.predict(img_array)
    distance = np.linalg.norm(emb - mean_embedding)
    return distance < threshold, distance

# ---------------- TEST ON NEW IMAGES ----------------
test_images = [
    "E:/Final_year_project/not_brain/animals_0.jpg",
    "E:/Final_year_project/zDatasets/val/brain_mri/1333.jpg"
]

for img_path in test_images:
    brain, score = is_brain(img_path)
    print(f"Image: {os.path.basename(img_path)} | Brain MRI: {brain} | Score: {score:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Image: animals_0.jpg | Brain MRI: True | Score: 0.1250
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Image: 1333.jpg | Brain MRI: True | Score: 0.0734


In [21]:
import os
import requests

PEXELS_API_KEY = "bJT0U4PIJxV1tSAtBR8r00k0Kf3fN1aGqviV590iuxOWRRMFNi639o7Z"

# flat list of all keywords
keywords = [
    "car vehicle", "street city", "food meal", "fruit vegetable",
    "mountain river", "ocean beach", "computer laptop", "office desk",
    "bird wildlife", "flower garden"
]

def fetch_images_pexels_flat(keywords, count_per_keyword, save_folder):
    headers = {"Authorization": PEXELS_API_KEY}
    os.makedirs(save_folder, exist_ok=True)
    
    for keyword in keywords:
        print(f"Fetching images for keyword: {keyword}")
        per_page = min(count_per_keyword, 80)  # Pexels max per page ~80
        page = 1
        total_fetched = 0
        
        while total_fetched < count_per_keyword:
            params = {"query": keyword, "per_page": per_page, "page": page}
            resp = requests.get("https://api.pexels.com/v1/search", headers=headers, params=params)
            resp.raise_for_status()
            data = resp.json().get("photos", [])
            if not data:
                break
            
            for photo in data:
                image_url = photo["src"]["original"]
                filename = os.path.join(save_folder, f"{keyword.replace(' ', '_')}_{page}_{photo['id']}.jpg")
                try:
                    with open(filename, "wb") as f:
                        f.write(requests.get(image_url).content)
                    total_fetched += 1
                except Exception as e:
                    print(f"Failed to download {image_url}: {e}")
                
                if total_fetched >= count_per_keyword:
                    break
            page += 1

# Example usage:
fetch_images_pexels_flat(keywords, count_per_keyword=50, save_folder="non_mri")



Fetching images for keyword: car vehicle
Fetching images for keyword: street city
Fetching images for keyword: food meal
Fetching images for keyword: fruit vegetable
Fetching images for keyword: mountain river
Fetching images for keyword: ocean beach
Fetching images for keyword: computer laptop
Fetching images for keyword: office desk
Fetching images for keyword: bird wildlife
Fetching images for keyword: flower garden


In [40]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight

# ---------------------------
# Config
# ---------------------------
train_dir = "zDatasets/train"
val_dir = "zDatasets/val"
img_size = (128, 128)
batch_size = 32
epochs = 5
learning_rate = 1e-4
model_save_path = "brain_filter_model.h5"

# ---------------------------
# Data Preparation
# ---------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.1
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# ---------------------------
# Compute Class Weights
# ---------------------------
classes = train_generator.classes
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(classes),
    y=classes
)
class_weights_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weights_dict}")

# ---------------------------
# Model Setup
# ---------------------------
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=img_size + (3,))
base_model.trainable = False  # freeze base layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ---------------------------
# Train top layers only
# ---------------------------
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    class_weight=class_weights_dict
)

# ---------------------------
# Optional: Safe fine-tuning
# ---------------------------
# Only unfreeze last 50 layers to avoid accuracy collapse
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=learning_rate/10),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    class_weight=class_weights_dict
)

# ---------------------------
# Save the model
# ---------------------------
model.save(model_save_path)
print(f"Model saved at {model_save_path}")


Found 5224 images belonging to 2 classes.
Found 2043 images belonging to 2 classes.
Class weights: {0: 0.8637566137566137, 1: 1.1872727272727273}


C:\Users\suhas\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 299s 2s/step - accuracy: 0.9023 - loss: 0.2167 - val_accuracy: 0.9971 - val_loss: 0.0182
Epoch 2/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.9965 - loss: 0.0119 - val_accuracy: 0.9971 - val_loss: 0.0158
Epoch 3/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.9991 - loss: 0.0049 - val_accuracy: 0.9966 - val_loss: 0.0170
Epoch 4/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 244s 1s/step - accuracy: 0.9989 - loss: 0.0050 - val_accuracy: 0.9971 - val_loss: 0.0137
Epoch 5/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.9992 - loss: 0.0028 - val_accuracy: 0.9976 - val_loss: 0.0094
Epoch 1/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 284s 2s/step - accuracy: 0.5839 - loss: 1.8891 - val_accuracy: 0.9990 - val_loss: 0.0017
Epoch 2/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 265s 2s/step - accuracy: 0.9769 - loss: 0.0766 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 3/5
164/164 ━━━━━━━━━━━━━━━━━━━━ 291s 2s/step - accuracy: 0.9932 - loss: 0.0259 - val_accuracy: 1.

Model saved at brain_filter_model.h5


In [31]:
import os

train_dir = "zDatasets/train"

for folder_name in os.listdir(train_dir):
    folder_path = os.path.join(train_dir, folder_name)
    if os.path.isdir(folder_path):
        num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
        print(f"{folder_name}: {num_files} images")


brain_mri: 3024 images
non_mri: 2200 images


In [49]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2

# ---------------------------
# Config
# ---------------------------
model_path = "brain_filter_model.h5"  # your saved model
img_size = (128, 128)                 # same as training size
test_images = ["E:/Final_year_project/zDatasets/val/non_mri/bird1.jpg",
               "E:/Final_year_project/zDatasets/val/non_mri/bird2.jpg",
               "E:/Final_year_project/zDatasets/val/non_mri/bird3.jpg"
               
               ]  # replace with your image paths

# ---------------------------
# Load the model
# ---------------------------
model = tf.keras.models.load_model(model_path)

# ---------------------------
# Helper function: preprocess single image
# ---------------------------
def preprocess_image(img_path, img_size):
    # Load image in BGR (OpenCV) and convert to RGB
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize
    img = cv2.resize(img, img_size)
    
    # Normalize
    img = img / 255.0
    
    # Expand dims to create batch size 1
    img = np.expand_dims(img, axis=0)
    return img

# ---------------------------
# Predict each image
# ---------------------------
for img_path in test_images:
    img = preprocess_image(img_path, img_size)
    pred = model.predict(img)
       # Since it's binary, threshold at 0.5
    print(pred)
    if pred[0][0] >= 0.5:
       print(f"{img_path}: Non-MRI (Confidence: {pred[0][0]:.3f})") 
    else:
       print(f"{img_path}: Brain MRI (Confidence: {1-pred[0][0]:.3f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.9999999]]
E:/Final_year_project/zDatasets/val/non_mri/bird1.jpg: Non-MRI (Confidence: 1.000)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
[[0.9999993]]
E:/Final_year_project/zDatasets/val/non_mri/bird2.jpg: Non-MRI (Confidence: 1.000)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[[1.]]
E:/Final_year_project/zDatasets/val/non_mri/bird3.jpg: Non-MRI (Confidence: 1.000)


In [43]:
print(train_generator.class_indices)


{'brain_mri': 0, 'non_mri': 1}


In [50]:
model.save("brain_filter_model.keras")


In [1]:
import wfdb

# Load a single ECG file
record = wfdb.rdrecord('files/p1102/p11021643/s46071360/46071360')

# Access signal data
signals = record.p_signal   # shape: [samples, leads]

# Get sampling rate
fs = record.fs


FileNotFoundError: [Errno 2] No such file or directory: 'e:/Final_year_project/files/p1102/p11021643/s46071360/46071360.hea'

In [ ]:
!file brain_filter_model.keras
!ls -l brain_filter_model.keras


'file' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import tensorflow as tf 
print(tf.__version__)
import keras 
print(keras.__version__)

2.19.0
3.10.0
